<a href="https://colab.research.google.com/github/thanhlong1997/100-nlp-papers/blob/master/autocorrect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf

# tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time
import tensorflow as tf

print(tf.__version__)

2.3.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
with open('/content/drive/MyDrive/AI_COLAB/Viettel/data_noise.txt','r',encoding='utf-8') as file:
  data_noise = file.read().split('\n')
with open('/content/drive/MyDrive/AI_COLAB/Viettel/data_true.txt','r',encoding='utf-8') as file:
  data_true = file.read().split('\n')

In [ ]:
def unfold_vnese(token):
    accent_dictionary = {
    'á':'as', 'à':'af', 'ả':'ar', 'ã':'ax', 'ạ':'aj', 'â':'aa', 'ấ':'aas', 'ầ':'aaf', 'ẩ':'aar', 'ẫ':'aax', 'ậ':'aaj', 'ă':'aw', 'ắ':'aws', 'ằ':'awf', 'ẳ':'awr', 'ẵ':'awx', 'ặ':'awj',
    'ó':'os', 'ò':'of', 'ỏ':'or', 'õ':'ox', 'ọ':'oj', 'ô':'oo', 'ố':'oos', 'ồ':'oof', 'ổ':'oor', 'ỗ':'oox', 'ộ':'ooj', 'ơ':'ow', 'ớ':'ows', 'ờ':'owf', 'ở':'owr', 'ỡ':'owx', 'ợ':'owj',
    'é':'es', 'è':'ef', 'ẻ':'er', 'ẽ':'ex', 'ẹ':'ej', 'ê':'ee', 'ế':'ees', 'ề':'eef', 'ể':'eer', 'ễ':'eex', 'ệ':'eej',
    'ú':'us', 'ù':'uf', 'ủ':'ur', 'ũ':'ux', 'ụ':'uj', 'ư':'uw', 'ứ':'uws', 'ừ':'uwf', 'ử':'uwr', 'ữ':'uwx', 'ự':'uwj',
    'í':'is', 'ì':'if', 'ỉ':'ir', 'ĩ':'ix', 'ị':'ij',
    'ý':'ys', 'ỳ':'yf', 'ỷ':'yr', 'ỹ':'yx', 'ỵ':'yj',
    'đ':'dd'
    }
    unfold = ''
    for character in token :
        if character in accent_dictionary.keys():
            unfold += accent_dictionary[character]
        else:
            unfold += character
    return unfold

In [ ]:
with open('combine_dataaaaaaaaaa.txt','w',encoding='utf-8') as file:
  for index in range(len(data_noise)):
    file.write(data_noise[index] +'\t' + data_true[index]+'\n')

In [ ]:
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://download.tensorflow.org/data/spa-eng.zip', 
    extract=True)

path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"

In [ ]:
path_to_file = '/content/drive/MyDrive/AI_COLAB/Viettel/combine_data.txt'

In [ ]:
path_to_file = '/content/drive/MyDrive/AI_COLAB/Viettel/combine_data.txt'

In [ ]:
with open(path_to_file,'r',encoding ='utf-8',buffering=100000) as file:
    data = file.read()

In [ ]:
data = data.split('\n')

In [ ]:
print(len(data))

5946867


In [ ]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    w = w.lower().strip()
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    # w = re.sub(r"([?.!,¿])", r" \1 ", w)
    # w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    # w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    # w = '<start> ' + w + ' <end>'
    return w

In [ ]:
def create_dataset(path, num_examples):
    lines = open(path, encoding='UTF-8').read().strip().split('\n')
    lines = lines[:-1]
    print(len(lines))
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    
    return word_pairs

In [ ]:
class LanguageSourceIndex():
  def __init__(self, lang):
    self.lang = lang
    self.word2idx = {}
    self.idx2word = {}
    self.vocab = set()
    # self.type_tokenizer = type_tokenizer
    
    self.create_index()
    # print(self.vocab)
  def create_index(self):
    # if self.type_tokenizer == 0:
    for phrase in self.lang:
      self.vocab.update(phrase)
    
    
    self.vocab = sorted(self.vocab)
    
    self.word2idx['<pad>'] = 0
    self.word2idx['<start>'] = 1
    self.word2idx['<end>'] = 2
    for index, word in enumerate(self.vocab):
      self.word2idx[word] = len(self.word2idx)
    
    self.idx2word = {v: k for k, v in self.word2idx.items()}

In [ ]:
class LanguageTargetIndex():
  def __init__(self, lang):
    self.lang = lang
    self.word2idx = {}
    self.idx2word = {}
    self.vocab = set()
    
    self.create_index()
    
  def create_index(self):
    # self.vocab.append('<pad>')
    # self.vocab.append('<start>')
    # self.vocab.append('<end>')
    for phrase in self.lang:
      self.vocab.update(phrase.split(' '))
    self.vocab = sorted(self.vocab)
    print(self.vocab)
    self.word2idx['<pad>'] = 0
    self.word2idx['<start>'] = 1
    self.word2idx['<end>'] = 2
    for index, word in enumerate(self.vocab):
      self.word2idx[word] = len(self.word2idx)
    
    self.idx2word = {v: k for k, v in self.word2idx.items()}

In [ ]:
a = LanguageTargetIndex('đường 1 bình chánh hồ chí minh')
a.word2idx


['', '1', 'b', 'c', 'g', 'h', 'i', 'm', 'n', 'á', 'ì', 'í', 'đ', 'ư', 'ồ', 'ờ']


{'': 3,
 '1': 4,
 '<end>': 2,
 '<pad>': 0,
 '<start>': 1,
 'b': 5,
 'c': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'm': 10,
 'n': 11,
 'á': 12,
 'ì': 13,
 'í': 14,
 'đ': 15,
 'ư': 16,
 'ồ': 17,
 'ờ': 18}

In [ ]:
def max_length(tensor):
    return max(len(t) for t in tensor)


def load_dataset(path, num_examples):
    # creating cleaned input, output pairs
    pairs = create_dataset(path, num_examples)
    print(pairs[0])
    # index language using the class defined above    
    inp_lang = LanguageSourceIndex(noise for noise, true in pairs)
    targ_lang = LanguageTargetIndex(true for noise, true in pairs)
    print(targ_lang.word2idx)
    
    
    
    input_tensor = []
    target_tensor = []
    for noise, true in pairs:
      noise_senten_tensor = [inp_lang.word2idx['<start>']]
      for s in noise:
        # if s in inp_lang.word2idx:
          noise_senten_tensor.append(inp_lang.word2idx[s])
      noise_senten_tensor.append(inp_lang.word2idx['<end>'])

      true_senten_tensor = [targ_lang.word2idx['<start>']]
      for s in true.split(' '):
        # if s in targ_lang.word2idx:
        true_senten_tensor.append(targ_lang.word2idx[s])
      true_senten_tensor.append(targ_lang.word2idx['<end>'])

      input_tensor.append(noise_senten_tensor)
      target_tensor.append(true_senten_tensor)
    
    
    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    print(max_length_inp,max_length_tar)
    print(len(input_tensor),len(target_tensor))
    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                                 maxlen=max_length_inp,
                                                                 padding='post')
    
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                                  maxlen=max_length_tar, 
                                                                  padding='post')
    
    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar

In [ ]:
num_examples = -1
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(path_to_file, num_examples)

5946865
['duongdwf 1 binhf chanhs hoof chis minh', 'đường 1 bình chánh hồ chí minh']
['', '-', '01-39', '03', '04', '05', '06', '07', '09', '1', '1-1', '1-10', '1-12', '1-19', '1-2', '1-2-3', '1-22', '1-3', '1-4', '1-42', '1-5', '1-6', '1-7', '1-8', '1-9', '10', '10-11', '10-14', '10-22', '100', '100a', '101', '1011', '102', '1021', '1022', '1025', '103', '104', '1041', '104a', '106', '107', '108', '1088', '109', '10a', '10b', '10c', '10d', '10e', '10f', '10g', '11', '11-6', '11-7', '110', '1106', '1107', '111', '112', '114', '115', '116', '117', '118', '119', '11a', '11b', '11c', '11d', '11e', '11l', '11n', '12', '12-52', '120', '121', '122', '123', '1234', '123a', '124', '126', '127d', '128', '128c', '129', '12a', '12ab', '12b', '12c', '12d', '12e', '13', '130', '131', '1315', '132', '135', '137', '138', '13a', '13b', '13c', '14', '14-1', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149/33', '14a', '14b', '14c', '14d', '14e', '14f', '15', '15-23', '150', '152', '15

In [ ]:
from numpy import save
save('/content/drive/MyDrive/AI_COLAB/Viettel/input_tensor.npy', input_tensor)

In [ ]:
from numpy import load
input_tensor = load('/content/drive/MyDrive/AI_COLAB/Viettel/input_tensor.npy')

In [ ]:
from numpy import load
target_tensor = load('/content/drive/MyDrive/AI_COLAB/Viettel/target_tensor.npy')

In [ ]:
input_tensor.shape

(5946864, 72)

In [ ]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)


(4757491, 4757491, 1189373, 1189373)

In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 128
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor, target_tensor)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
iterator = iter(dataset)

In [ ]:
vocab_tar_size

2541

In [ ]:
print(vocab_inp_size)

51


In [ ]:
def gru(units,is_backward):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
  if tf.test.is_gpu_available():
    return tf.keras.layers.GRU(units, 
                                    return_sequences=True, 
                                    return_state=True, 
                                    recurrent_initializer='glorot_uniform',
                                    go_backwards = is_backward)
  else:
    return tf.keras.layers.GRU(units, 
                               return_sequences=True, 
                               return_state=True, 
                               recurrent_activation='sigmoid', 
                               recurrent_initializer='glorot_uniform',
                               go_backwards = is_backward)


In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru_forward = gru(self.enc_units,is_backward= False)
        self.gru_backward = gru(self.enc_units,is_backward = True)
    def call(self, x, hidden):
        x = self.embedding(x)
        # print('after embeding',x.shape)
        output_f, state_f = self.gru_forward(x, initial_state = hidden)     
        output_b, state_b = self.gru_backward(x, initial_state = hidden)
        # print(output_f.shape) 
        # print(state_f.shape) 
        return  tf.keras.layers.Concatenate()([output_f, output_b]), state_f
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
def split_list(x,key):
  result = []
  sub_list = []
  for item in x :
    if item != key:
      sub_list.append(item)
    else:
      result.append(np.asarray(sub_list))
      sub_list = []
  return result

In [ ]:
split_list(np.array([1,2,3,4,5,3,4,3,7]),3)

[array([1, 2]), array([4, 5]), array([4])]

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru_char_forward = gru(self.enc_units,is_backward= False)
        self.gru_char_backward = gru(self.enc_units,is_backward = True)
        self.gru_word_forward = gru(self.enc_units,is_backward= False)
        self.gru_word_backward = gru(self.enc_units,is_backward = True)
    def call(self, x, hidden):
        word_split = split_list(x,inp_lang.word2idx[' '])
        print(word_split.shape)
        x = self.embedding(x)
        print(x.shape)
        senten = []
        for word in word_split:
          x =  self.embedding(word)
          print(x.shape)
          output_char_f, state_char_f = self.gru_char_forward(x, initial_state = hidden)     
          output_char_b, state_char_b = self.gru_char_backward(x, initial_state = hidden)  
          senten.append(Concatenate()([output_char_f, output_char_b]))

        senten = np.asarray(senten)
        print(senten.shape)
        output_word_f, state_word_f = self.gru_word_forward(senten, initial_state = hidden)     
        output_word_b, state_word_b = self.gru_word_backward(senten, initial_state = hidden) 
        print(output_word_f.shape)
        print(state_word_f)
        return  Concatenate()([output_word_f, output_word_b]), state_word_f

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units, is_backward=False)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [ ]:
tf.config.list_physical_devices('GPU')
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [ ]:
optimizer =  tf.optimizers.Adam()


def loss_function(real, pred):
  mask = 1 - np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
  return tf.reduce_mean(loss_)

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/AI_COLAB/Viettel/training_checkpoints'
checkpoint_dir2 = '/content/drive/MyDrive/AI_COLAB/Viettel/training_checkpoints2'
checkpoint_prefix = os.path.join(checkpoint_dir2, "ckpt")
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer,  iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, checkpoint_dir, max_to_keep=3)

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
EPOCHS = 20
tf.debugging.set_log_device_placement(True)
ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
else:
    print("Initializing from scratch.")
    
for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    # for (batch, (inp, targ)) in enumerate(dataset):
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
    
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)  
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        ckpt.step.assign_add(1)
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                          batch,
                                                          batch_loss.numpy()))
            save_path = manager.save()
            print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
            print("loss {:1.2f}".format(loss.numpy()))

    # saving (checkpoint) the model every 2 epochs
    # if (epoch + 1) % 2 == 0:
    # checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Restored from /content/drive/MyDrive/AI_COLAB/Viettel/training_checkpoints/ckpt-3
Epoch 1 Batch 0 Loss 1.1895
Saved checkpoint for step 103: /content/drive/MyDrive/AI_COLAB/Viettel/training_checkpoints/ckpt-4
loss 17.84
Epoch 1 Batch 100 Loss 1.2005
Saved checkpoint for step 203: /content/drive/MyDrive/AI_COLAB/Viettel/training_checkpoints/ckpt-5
loss 18.01
Epoch 1 Batch 200 Loss 1.1126
Saved checkpoint for step 303: /content/drive/MyDrive/AI_COLAB/Viettel/training_checkpoints/ckpt-6
loss 16.69
Epoch 1 Batch 300 Loss 1.0610
Saved checkpoint for step 403: /content/drive/MyDrive/AI_COLAB/Viettel/training_checkpoints/ckpt-7
loss 15.91
Epoch 1 Batch 400 Loss 1.0775
Saved checkpoint for step 503: /content/drive/MyDrive/AI_COLAB/Viettel/training_checkpoints/ckpt-8
loss 16.16
Epoch 1 Batch 500 Loss 1.0009
Saved checkpoint for step 603: /content/drive/MyDrive/AI_COLAB/Viettel/training_checkpoints/ckpt-9
loss 15.01
Epoch 1 Batch 600 Loss 1.0994
Saved checkpoint for step 703: /content/drive/MyDr

KeyboardInterrupt: ignored

In [ ]:
(next(iterator)[0])

<tf.Tensor: shape=(128, 72), dtype=int32, numpy=
array([[ 1, 23, 28, ...,  0,  0,  0],
       [ 1, 23, 23, ...,  0,  0,  0],
       [ 1, 23, 40, ...,  0,  0,  0],
       ...,
       [ 1, 23, 40, ...,  0,  0,  0],
       [ 1, 23, 23, ...,  0,  0,  0],
       [ 1, 23, 23, ...,  0,  0,  0]], dtype=int32)>

In [ ]:
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
    sentence = preprocess_sentence(sentence)

    # inputs = [inp_lang.word2idx[i] for i in sentence]
    inputs = [inp_lang.word2idx['<start>']]
    for s in sentence:
      inputs.append(inp_lang.word2idx[s])
    inputs.append(inp_lang.word2idx['<end>'])
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(len(sentence.split(' '))):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weigths to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()
        print(predictions.shape)
        arr = np.array(predictions[0])
        print(arr[1725])
        a = arr.argsort()[-10:][::-1]
        print(a,arr[a])
        # a = tf.make_ndarray(predictions)
        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    
    fontdict = {'fontsize': 14}
    
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    plt.show()

def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    result, sentence, attention_plot = evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
        
    print('Input: {}'.format(sentence))
    print('Predicted translation: {}'.format(result))
    
    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    # plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
translate(unfold_vnese('Traanf hueng dao'), encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

after embeding (1, 72, 256)
(1, 72, 1024)
(1, 1024)
(1, 2541)
-1.23448
[2485 2393 1308 2100 1632 1398 1526 1861 1409 1369] [20.935604 18.038221 12.926941 12.070888 11.143383 10.232903  9.701294
  8.755151  8.75013   8.615571]
(1, 2541)
-0.5791761
[1529 1049 1856 1436 1533 1051  965 1887 1070 1403] [17.338915  14.424824  13.287314  13.0891    13.024799  12.601821
 11.8564625 11.616872  11.506042  11.209258 ]
(1, 2541)
-2.1797726
[1258    2  919 1710  867 2405    5 1324 1718 1721] [20.580462 14.682459 14.279002 12.017232 11.483554 11.420092 11.229128
 11.067651 10.968297 10.799262]
Input: traanf hueng dao
Predicted translation: đĩnh liên ghe 


In [ ]:
print(targ_lang.word2idx['trần'])

2209


In [ ]:
targ_lang.idx2word[2485]

'đĩnh'

In [ ]:
def unfold_vnese(token):
    token = token.lower()
    accent_dictionary = {
    'á':'as', 'à':'af', 'ả':'ar', 'ã':'ax', 'ạ':'aj', 'â':'aa', 'ấ':'aas', 'ầ':'aaf', 'ẩ':'aar', 'ẫ':'aax', 'ậ':'aaj', 'ă':'aw', 'ắ':'aws', 'ằ':'awf', 'ẳ':'awr', 'ẵ':'awx', 'ặ':'awj',
    'ó':'os', 'ò':'of', 'ỏ':'or', 'õ':'ox', 'ọ':'oj', 'ô':'oo', 'ố':'oos', 'ồ':'oof', 'ổ':'oor', 'ỗ':'oox', 'ộ':'ooj', 'ơ':'ow', 'ớ':'ows', 'ờ':'owf', 'ở':'owr', 'ỡ':'owx', 'ợ':'owj',
    'é':'es', 'è':'ef', 'ẻ':'er', 'ẽ':'ex', 'ẹ':'ej', 'ê':'ee', 'ế':'ees', 'ề':'eef', 'ể':'eer', 'ễ':'eex', 'ệ':'eej',
    'ú':'us', 'ù':'uf', 'ủ':'ur', 'ũ':'ux', 'ụ':'uj', 'ư':'uw', 'ứ':'uws', 'ừ':'uwf', 'ử':'uwr', 'ữ':'uwx', 'ự':'uwj',
    'í':'is', 'ì':'if', 'ỉ':'ir', 'ĩ':'ix', 'ị':'ij',
    'ý':'ys', 'ỳ':'yf', 'ỷ':'yr', 'ỹ':'yx', 'ỵ':'yj',
    'đ':'dd'
    }
    posible = []
    unfold = ''
    accent = []
    for character in token :
        if character in accent_dictionary.keys():
            unfold += accent_dictionary[character][0]
            for char in accent_dictionary[character][1:]:
                if char not in accent:
                    accent.append(char)
        else:
            unfold += character
    print(accent)
    unfold = unfold + ''.join(x for x in accent)
    posible.append(unfold)
    
    unfold = ''
    for character in token :
        if character in accent_dictionary.keys():
            unfold += accent_dictionary[character]
        else:
            unfold += character
    if unfold not in posible:
        posible.append(unfold)
    return posible

In [ ]:
import numpy as np

letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
           'n','o','p','q','r','s','t','u','v','w','x','y','z',]

def noise_maker(sentence, threshold):   
    token_list = sentence.split(' ')
    unfold_tokens = []
    for i in token_list:
        unfold_tokens.append(unfold_vnese(i))
    print(unfold_tokens)
#     1/0
    sentences=['']
    for tokens in unfold_tokens:
        print(sentences)
        for token in tokens:
            for senten in sentences:
                print(sentences)
                a = senten+ ' ' + token
                buffer.append(a)
            sentences = buffer
    
    1/0
    noisy_sentence = []
    i = 0
    replace_dict = {'q':['w'],'w':['q','e'],'e':['w','r'],'r':['e','t'],'t':['r','y'],'y':['t','u'],'u':['y','i'],'i':['u','o'],'o':['i','p'],'p':['o','['],
                    'a':['s'],'s':['a','d'],'d':['s','f'],'f':['d','g'],'g':['f','h'],'h':['g','j'],'j':['h','k'],'k':['j','l'],'l':['k',';'],';':['l','\''],
                    'z':['x'],'x':['z','c'],'c':['x','v'],'v':['c','b'],'b':['v','n'],'n':['b','m'],'m':['n',',']}
    while i < len(sentence):
        random = np.random.uniform(0,1,1)
        # Most characters will be correct since the threshold value is high
        if random < threshold:
            noisy_sentence.append(sentence[i])
        else:
            new_random = np.random.uniform(0,1,1)
            # ~33% chance characters will swap locations
            if new_random > 0.67:
                if i == len(sentence) - 1:
                    # If last character in sentence, it will not be typed
                    continue
                else:
                    # if any other character, swap order with following character
                    if sentence[i] != ' ' and sentence[i+1] != ' ':
                        noisy_sentence.append(sentence[i+1])
                        noisy_sentence.append(sentence[i])
                        i += 1
            # ~33% chance an extra lower case letter will be added to the sentence
            elif new_random < 0.33:
                if sentence[i] in replace_dict.keys():
                    random_letter = np.random.choice(replace_dict[sentence[i]], 1)[0]
                    noisy_sentence.append(random_letter)
#                 print(random_letter)
                else:
                    noisy_sentence.append(sentence[i])
            # ~33% chance a character will not be typed
            else:
                if sentence[i] == ' ':
                    noisy_sentence.append(sentence[i])
                else:
                    pass     
        i += 1
    return ''.join(x for x in noisy_sentence)

In [ ]:
print(noise_maker('hồ chúng minh',0.9))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: ignored